# 한국고전종합DB 번역문 스크래핑(Selenium)

In [ ]:
!pip install --upgrade pandas
!pip install -U webdriver_manager selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install lxml
!pip install BeautifulSoup4
!pip install pickle5

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'apt'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Running setup.py clean for pickle5
Failed to build pickle5
    Running setup.py install for pickle5: started
    Running setup.py install for pickle5: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Administrator\miniconda3\envs\python38\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-lx4f3qf8\\pickle5_1d56fd7268a543bbad6dccd829e75841\\setup.py'"'"'; __file__='"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-lx4f3qf8\\pickle5_1d56fd7268a543bbad6dccd829e75841\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-wheel-mwcmqzk4'
       cwd: C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-lx4f3qf8\pickle5_1d56fd7268a543bbad6dccd829e75841\
  Complete output (36 lines):
  running bdist_wheel
  running build
  run

In [ ]:
import requests
import pandas as pd
import lxml
import xml.etree.ElementTree as ET
from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import re
import random
import time

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('C:/Users/Administrator/Documents/Downloads/chromedriver.exe', options=chrome_options)
# driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

<ipython-input-2-e633f5096df4>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Administrator/Documents/Downloads/chromedriver.exe', options=chrome_options)


### 자료 ID 스크래핑

### 자료ID pickle 파일 읽기

In [ ]:
with open('G:/공유 드라이브/ProjectOuri/DB&Code/한국고전종합DB/한국문집총간_asso.pickle', "rb") as fh:
  data = pd.read_pickle(fh)

In [ ]:
data

,id,type,연계시작,연계종료
0,ITKC_MO_0001A_0010_010_0010,이미지,ITKC_MO_0001A_A001_003A,ITKC_MO_0001A_A001_003D
1,ITKC_MO_0001A_0010_010_0010,번역문,ITKC_BT_0001A_0010_000_0010,ITKC_BT_0001A_0010_000_0010
2,ITKC_MO_0001A_0010_010_0020,이미지,ITKC_MO_0001A_A001_004A,ITKC_MO_0001A_A001_005A
3,ITKC_MO_0001A_0010_010_0020,번역문,ITKC_BT_0001A_0010_000_0020,ITKC_BT_0001A_0010_000_0020
4,ITKC_MO_0001A_0010_010_0030,이미지,ITKC_MO_0001A_A001_005C,ITKC_MO_0001A_A001_006A
...,...,...,...,...
782544,ITKC_MO_1252A_0290_000_0010,번역문,ITKC_BT_1252A_0290_000_0010,ITKC_BT_1252A_0290_000_0010
782545,ITKC_MO_1252A_0290_000_0020,이미지,ITKC_MO_1252A_B144_713C,ITKC_MO_1252A_B144_713D
782546,ITKC_MO_1252A_0290_000_0020,번역문,ITKC_BT_1252A_0290_000_0020,ITKC_BT_1252A_0290_000_0020
782547,ITKC_MO_1252A_0290_000_0030,이미지,ITKC_MO_1252A_B144_713D,ITKC_MO_1252A_B144_714A


In [ ]:
to_ko = data[data['type']=='번역문']
to_ko

,id,type,연계시작,연계종료
1,ITKC_MO_0001A_0010_010_0010,번역문,ITKC_BT_0001A_0010_000_0010,ITKC_BT_0001A_0010_000_0010
3,ITKC_MO_0001A_0010_010_0020,번역문,ITKC_BT_0001A_0010_000_0020,ITKC_BT_0001A_0010_000_0020
5,ITKC_MO_0001A_0010_010_0030,번역문,ITKC_BT_0001A_0010_000_0030,ITKC_BT_0001A_0010_000_0030
7,ITKC_MO_0001A_0020_010_0010,번역문,ITKC_BT_0001A_0020_010_0010,ITKC_BT_0001A_0020_010_0010
9,ITKC_MO_0001A_0020_010_0020,번역문,ITKC_BT_0001A_0020_010_0020,ITKC_BT_0001A_0020_010_0020
...,...,...,...,...
782540,ITKC_MO_1252A_0270_000_0040,번역문,ITKC_BT_1252A_0270_000_0040,ITKC_BT_1252A_0270_000_0040
782542,ITKC_MO_1252A_0280_000_0010,번역문,ITKC_BT_1252A_0280_000_0010,ITKC_BT_1252A_0280_000_0010
782544,ITKC_MO_1252A_0290_000_0010,번역문,ITKC_BT_1252A_0290_000_0010,ITKC_BT_1252A_0290_000_0010
782546,ITKC_MO_1252A_0290_000_0020,번역문,ITKC_BT_1252A_0290_000_0020,ITKC_BT_1252A_0290_000_0020


In [ ]:
#겹치는 것 없음
to_ko.drop_duplicates(['id'], keep ='first')

,id,type,연계시작,연계종료
1,ITKC_MO_0001A_0010_010_0010,번역문,ITKC_BT_0001A_0010_000_0010,ITKC_BT_0001A_0010_000_0010
3,ITKC_MO_0001A_0010_010_0020,번역문,ITKC_BT_0001A_0010_000_0020,ITKC_BT_0001A_0010_000_0020
5,ITKC_MO_0001A_0010_010_0030,번역문,ITKC_BT_0001A_0010_000_0030,ITKC_BT_0001A_0010_000_0030
7,ITKC_MO_0001A_0020_010_0010,번역문,ITKC_BT_0001A_0020_010_0010,ITKC_BT_0001A_0020_010_0010
9,ITKC_MO_0001A_0020_010_0020,번역문,ITKC_BT_0001A_0020_010_0020,ITKC_BT_0001A_0020_010_0020
...,...,...,...,...
782540,ITKC_MO_1252A_0270_000_0040,번역문,ITKC_BT_1252A_0270_000_0040,ITKC_BT_1252A_0270_000_0040
782542,ITKC_MO_1252A_0280_000_0010,번역문,ITKC_BT_1252A_0280_000_0010,ITKC_BT_1252A_0280_000_0010
782544,ITKC_MO_1252A_0290_000_0010,번역문,ITKC_BT_1252A_0290_000_0010,ITKC_BT_1252A_0290_000_0010
782546,ITKC_MO_1252A_0290_000_0020,번역문,ITKC_BT_1252A_0290_000_0020,ITKC_BT_1252A_0290_000_0020


In [ ]:
ko_list=to_ko['id'].to_list()
ko_list

['ITKC_MO_0001A_0010_010_0010',
 'ITKC_MO_0001A_0010_010_0020',
 'ITKC_MO_0001A_0010_010_0030',
 'ITKC_MO_0001A_0020_010_0010',
 'ITKC_MO_0001A_0020_010_0020',
 'ITKC_MO_0001A_0020_010_0030',
 'ITKC_MO_0001A_0020_010_0040',
 'ITKC_MO_0001A_0020_010_0050',
 'ITKC_MO_0001A_0020_010_0060',
 'ITKC_MO_0001A_0020_010_0070',
 'ITKC_MO_0001A_0020_010_0080',
 'ITKC_MO_0001A_0020_010_0090',
 'ITKC_MO_0001A_0020_010_0100',
 'ITKC_MO_0001A_0030_010_0010',
 'ITKC_MO_0001A_0030_010_0020',
 'ITKC_MO_0001A_0030_010_0030',
 'ITKC_MO_0001A_0030_010_0040',
 'ITKC_MO_0001A_0030_010_0050',
 'ITKC_MO_0001A_0030_010_0060',
 'ITKC_MO_0001A_0030_010_0070',
 'ITKC_MO_0001A_0030_010_0080',
 'ITKC_MO_0001A_0030_010_0090',
 'ITKC_MO_0001A_0030_010_0100',
 'ITKC_MO_0001A_0040_010_0010',
 'ITKC_MO_0001A_0040_010_0020',
 'ITKC_MO_0001A_0040_010_0030',
 'ITKC_MO_0001A_0040_010_0040',
 'ITKC_MO_0001A_0040_010_0050',
 'ITKC_MO_0001A_0040_010_0060',
 'ITKC_MO_0001A_0040_010_0070',
 'ITKC_MO_0001A_0040_010_0080',
 'ITKC_M

In [ ]:
#만개 씩 나누기
df =pd.DataFrame()
start = 0
end = 10000
for i in range (0, 11):
  df=to_ko[start:end]
  df.to_pickle('한국문집총간_trans_{}.pickle'.format(i+1))

  start+=10000
  end+=10000



['ITKC_MO_0001A_0010_010_0010',
 'ITKC_MO_0001A_0010_010_0020',
 'ITKC_MO_0001A_0010_010_0030',
 'ITKC_MO_0001A_0020_010_0010',
 'ITKC_MO_0001A_0020_010_0020']

### 자동화

In [ ]:

num = input('1~11까지 입력하세요 \n(ex : 1 = 0~9999, 2 = 10000~19999...)')

with open('G:/공유 드라이브/ProjectOuri/DB&Code/한국고전종합DB/한국문집총간_trans_{}.pickle'.format(num), "rb") as fh:
  data = pd.read_pickle(fh)
  
print(num)

1


In [ ]:
ko_list=data['id'].to_list()
ko_list

['ITKC_MO_0001A_0010_010_0010',
 'ITKC_MO_0001A_0010_010_0020',
 'ITKC_MO_0001A_0010_010_0030',
 'ITKC_MO_0001A_0020_010_0010',
 'ITKC_MO_0001A_0020_010_0020',
 'ITKC_MO_0001A_0020_010_0030',
 'ITKC_MO_0001A_0020_010_0040',
 'ITKC_MO_0001A_0020_010_0050',
 'ITKC_MO_0001A_0020_010_0060',
 'ITKC_MO_0001A_0020_010_0070',
 'ITKC_MO_0001A_0020_010_0080',
 'ITKC_MO_0001A_0020_010_0090',
 'ITKC_MO_0001A_0020_010_0100',
 'ITKC_MO_0001A_0030_010_0010',
 'ITKC_MO_0001A_0030_010_0020',
 'ITKC_MO_0001A_0030_010_0030',
 'ITKC_MO_0001A_0030_010_0040',
 'ITKC_MO_0001A_0030_010_0050',
 'ITKC_MO_0001A_0030_010_0060',
 'ITKC_MO_0001A_0030_010_0070',
 'ITKC_MO_0001A_0030_010_0080',
 'ITKC_MO_0001A_0030_010_0090',
 'ITKC_MO_0001A_0030_010_0100',
 'ITKC_MO_0001A_0040_010_0010',
 'ITKC_MO_0001A_0040_010_0020',
 'ITKC_MO_0001A_0040_010_0030',
 'ITKC_MO_0001A_0040_010_0040',
 'ITKC_MO_0001A_0040_010_0050',
 'ITKC_MO_0001A_0040_010_0060',
 'ITKC_MO_0001A_0040_010_0070',
 'ITKC_MO_0001A_0040_010_0080',
 'ITKC_M

In [ ]:
row_list = []
error_list =[]

for id in tqdm(ko_list) :
    try :
        url = f'https://db.itkc.or.kr/dir/node?dataId={id}&viewSync=TR'
        res = requests.get(url)
        time.sleep(random.uniform(3,6))
        req = driver.page_source
        soup = BeautifulSoup(res.text,'html.parser')

        
        ch_text = soup.select_one('div > div.text_body.ori > div').text
        ko_text = soup.select('.text_body')[1].text

        row_inter = {'ID': id, '원문':ch_text, '번역문':ko_text}
        
        row_list.append(row_inter)
    
    except AttributeError :
        try :
            ko_text = soup.select('.text_body')[1].text
            row_inter = {'ID': id, '원문':'원문 오류', '번역문':ko_text}
            row_list.append(row_inter)
            error = {'ID' : id, 'Error' :'원문'}
            error_list.append(error)
            print('오류난 id : '+ id)
        except IndexError :
            row_inter = {'ID': id, '원문': '원문 오류', '번역문':'번역문 오류'}
            row_list.append(row_inter)
            error = {'ID' : id, 'Error' :'번역문'}
            error_list.append(error)
            print('오류난 id : '+ id) 
    
    except IndexError :
        try :
            ch_text = soup.select_one('div > div.text_body.ori > div').text
            row_inter = {'ID': id, '원문': ch_text, '번역문':'번역문 오류'}
            row_list.append(row_inter)
            error = {'ID' : id, 'Error' :'번역문'}
            error_list.append(error)
            print('오류난 id : '+ id) 
        except AttributeError :
            row_inter = {'ID': id, '원문': '원문 오류', '번역문':'번역문 오류'}
            row_list.append(row_inter)
            error = {'ID' : id, 'Error' :'원문'}
            error_list.append(error)
            print('오류난 id : '+ id) 
    
    

# %%

df = pd.DataFrame(row_list)
df.to_pickle('translate {}.pickle'.format(num))

df_error = pd.DataFrame(error_list)
df_error.to_pickle('error {}.pickle'.format(num))

 25%|██▌       | 2527/9999 [3:22:41<10:55:45,  5.27s/it]

오류난 id : ITKC_MO_0005A_0040_020_0120


 30%|██▉       | 2977/9999 [3:58:56<9:36:41,  4.93s/it]

오류난 id : ITKC_MO_0011A_0170_010_0020


 88%|████████▊ | 8765/9999 [11:46:35<1:51:58,  5.44s/it]

...

 88%|████████▊ | 8772/9999 [11:47:09<1:42:28,  5.01s/it]

오류난 id : ITKC_MO_0024A_0160_080_0050


 88%|████████▊ | 8773/9999 [11:47:15<1:47:25,  5.26s/it]

오류난 id : ITKC_MO_0024A_0160_080_0060


100%|██████████| 9999/9999 [13:24:39<00:00,  4.83s/it]


In [ ]:
df = pd.DataFrame(row_list)
df.to_pickle('translate {}.pickle'.format(num))

df_error = pd.DataFrame(error_list)
df_error.to_pickle('error {}.pickle'.format(num))

#### 피클 데이터 합치기

In [ ]:
for i in range(1, 12):
    globals()['trans_{}'.format(i)] = pd.read_pickle('D:/고전종합DB/translate {}.pickle'.format(i))
    print(str(i)+'번 피클파일 완료')

'ITKC_MO_0011A_0170_010_0020'

In [ ]:
df_trans = pd.DataFrame()
for i in range (1, 12):
    df_part=pd.read_pickle('translate {}.pickle'.format(i))
    df_trans = pd.concat([df_trans, df_part], ignore_index=True)
    
df_trans